 # 4D-OBC Seed extraction
 
This notebook explains how to use the extraction-function of the py4dgeo-packag which extracts 4D-objects-by-change automatically from a point-cloud, by using either the Ramer-Douglas-Peucker-algorithm (RDP) or a Decision-Tree-Regressor (DTR).

Implemented in: `py4dgeo.extract`

**Author(s)** 

Scharnagl, Robin (Technische Universität München)

**Related publication**

 Ulm M, Elias M,  Eltner A, Lotsari E & Anders K. (2025). Automated change detection in photogrammetric 4D point clouds – transferability and extension of 4D objects‑by‑change 
for monitoring riverbank dynamics using low‑cost cameras. *Applied Geomatics*, https://doi.org/10.1007/s12518-025-00623-9   


## **Method description**
This section should provide a detailed technical explanation of the method:

- **Theoretical foundation**: To detect changes in a surface, this algorithm uses distances from a M3C2 point-cloud comparison to detect linear changes of the surface. 
Two different methods are available:

    - The Ramer–Douglas–Peucker (RDP) algorithm simplifies the time-series curve of each core point, dividing it into polygons (segments) that represent approximately linear trends.

    - The Decision Tree Regressor (DTR) estimates the local gradient (rate of change) of each time series and groups similar gradients together to form regions with nearly constant slope.

    Both approaches aim to find and quantify sections of linear change in the point cloud distance data.
- **Algorithm**: The alorithm detects linear temporal changes at each core-point in a  point cloud. If a linear change exceeds a user-defined threshold, it gets added to a list of seed candidates. 
    - **Ramer-Douglas-Peucker:** By using the RDP-method, the time-series of a corepoint gets divided into segments of (approximately) linear change. These segments are then expanded, as long as the amplitude exceeds the user-definded height-threshold, and the temporal length is shorter tahn the user-defined max_change_period. The valid segments are then stored as seeds with their intervals, which are then sorted according to their change magnitude.
    - **Decision Tree Regressor:** With the Decision-Tree-Regressor approach, the regressor splits the data into regions with similar values, which is the rate-of-change in this case. If the slope is roughly constant, the data behaves lineraly. Therefore, for each core point, teh gradient is calculated, which is then used by the Decision Tree Regressor, to create regions of constant slope. For each of these regions, a linear regression is fit, expanded, and filtered based on the amplitude threshold and the maximum duration. These are then sorted based on change magnitude.

- **Key parameters**: The parameters that the userr can define are the following:
    - `method` (str): selects the method, either 'RDP' or 'DTR' 
    - `smoothing_window` (int): defines the size of the window to smooth the data
    - `seed_subsampling` (int): reduces the number of seedpoints, that are taken into account
    - `max_change_period` (int): the maximum number of epochs for a segment-length
    - `neighborhood_radius` (float): defines the spatial neighbourhood used during region growing
    - `min_segments` (int): minimum number of epochs of each segment 
    - `max_segments` (int): maximum number of epoch of each segment 
    - `height_threshold` (float): minimum change required to be accepted as a seed
    - `data_gap` (int or None): forbids seeds crossing a known missing-data epoch


First, we download the dataset:

In [ ]:
# Load Required Package
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import pooch
import os 
from datetime import datetime
import py4dgeo 


# Load Dataset
# Download data from zenodo and set path to point cloud folder
p = pooch.Pooch(base_url="doi:10.5281/zenodo.10003574/", path=pooch.os_cache("py4dgeo"))
p.load_registry_from_doi()
p.fetch("module3.zip", processor=pooch.Unzip(), progressbar=True)
pc_dir = p.path / "module3.zip.unzip" / "module3" / "schneeferner" / "pointclouds"


We then create a SpatiotemporalAnalysis-Object from the dataset:

In [ ]:
# list of point clouds (time series)
pc_list = os.listdir(pc_dir)
pc_list[:5] # print the first elements


timestamps = []
for f in pc_list:
    if not f.endswith('.laz'):
        continue

    # get the timestamp from the file name
    timestamp_str = '_'.join(f.split('.')[0].split('_')[1:]) # yields YYMMDD_hhmmss

    # convert string to datetime object
    timestamp = datetime.strptime(timestamp_str, '%y%m%d_%H%M%S')
    timestamps.append(timestamp)

timestamps[:5]

analysis = py4dgeo.SpatiotemporalAnalysis(f'{pc_dir}\schneeferner.zip', force=True)
#%%
# specify the reference epoch
reference_epoch_file = os.path.join(pc_dir, pc_list[0])

# read the reference epoch and set the timestamp
reference_epoch = py4dgeo.read_from_las(reference_epoch_file)
reference_epoch.timestamp = timestamps[0]

# set the reference epoch in the spatiotemporal analysis object
analysis.reference_epoch = reference_epoch

After creating the SpatiotemporalAnalysis-object, we can simply add the extract-function and add parameters to the users-preference. 

In [ ]:
# Run the Method
analysis.extract(method = 'RDP', smoothing_window=5, seed_subsampling=1, max_change_period = 200, neighborhood_radius = 1, min_segments=10, max_segments=10000, height_threshold = 0.05)

## References
-  Ulm M, Elias M,  Eltner A, Lotsari E & Anders K. (2025). Automated change detection in photogrammetric 4D point clouds – transferability and extension of 4D objects‑by‑change 
for monitoring riverbank dynamics using low‑cost cameras. *Applied Geomatics*, pages. https://doi.org/10.1007/s12518-025-00623-9   